# Task 6 – Temporal Forecasting with LSTM

In this task, we build a sequence model (LSTM) to forecast short-term solar power output using recent historical measurements.

- **Target variable:** `DC_POWER`
- **Model type:** Sequence-to-one time series forecasting (LSTM)
- **Prediction horizon:** 1 hour ahead (4 × 15-minute steps)
- **Input window:** Previous 24 hours of data (96 × 15-minute steps)

We use the preprocessed dataset produced in the previous notebook (EDA & preprocessing), so we do not repeat merging or cleaning here.

#### IMPORTS AND CONFIGURATION

In [5]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Reproducibility
np.random.seed(42)
tf.random.set_seed(42)
SEQ_LENGTH = 96 # 24h of history (96 x 15 min)
HORIZON = 4 # 1h ahead (4 x 15 min)


ModuleNotFoundError: No module named 'tensorflow.python'

#### LOAD PROCESSED DATASET (TASK 2)

In [ ]:

#df = pd.read_csv("/mnt/data/processed_solar_data.csv", parse_dates=["DATE_TIME"])
#df = pd.read_pickle("/mnt/data/processed_solar_data.pkl")


df.head()

In [ ]:
# Ensure datetime is parsed
df["DATE_TIME"] = pd.to_datetime(df["DATE_TIME"])

# Sort chronologically
df = df.sort_values("DATE_TIME").reset_index(drop=True)


#### FEATURE SELECTION AND SCALING

In [ ]:
features = []

target_col = "DC_POWER"
target_idx = features.index(target_col)


In [ ]:
values = df[features].values.astype(float)

# Chronological split indices (on raw time series, before sequence creation)
n = len(values)
train_end = int(n * 0.7)
val_end   = int(n * 0.85)

train_vals = values[:train_end]
val_vals   = values[train_end:val_end]
test_vals  = values[val_end:]

In [ ]:
# Scale based on training data only, apply to val/test with same scaler
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_vals)
val_scaled   = scaler.transform(val_vals)
test_scaled  = scaler.transform(test_vals)

train_scaled.shape, val_scaled.shape, test_scaled.shape